In [2]:
import pip as pip
import pandas as pd
import geopandas as gpd
import openrouteservice as ors
import time

client = ors.Client(key= """eyJvcmciOiI1YjNjZTM1OTc4NTExMTAwMDFjZjYyNDgiLCJpZCI6IjhjOWMzNmU1YjE4MTRiZWI5YmY0ZjJiYmZmOWU2ZWE2IiwiaCI6Im11cm11cjY0In0=""") #you need an openrouteservice API-Key to run this script. further info is contained in the readme

In [3]:
Adress_list = pd.read_excel("data.xlsx")
Afterparty = pd.read_excel("Afterparty.xlsx")


In [4]:
Adress_list

,Team Nr.,Name 1,Name 2,Adress,Ring at,Allergies or else,will to double,vegan,routeplus_vegan,readiness starter
0,1,Lukas,Ilaija,"Im Neuenheimer Feld 133, 69120 Heidelberg",NaN,NaN,3,True,5.0,True
1,2,Muhammad,Ilayda,"Im Neuenheimer Feld 522, 69120 Heidelberg",NaN,NaN,5,False,NaN,True
2,3,Sahin,Wolodimr,"Klostergasse 3, 69123 Heidelberg",NaN,NaN,1,False,NaN,True
3,4,Kamala,Micha,"Elsterweg 5, 69123 Heidelberg",NaN,NaN,1,False,NaN,True
4,5,Joe,Donald,"Kranichweg 88, 69123 Heidelberg",NaN,NaN,1,True,3.0,True
5,6,Barak,Angela,"Danziger Straße 16, 69124 Heidelberg",NaN,NaN,1,True,10.0,True
6,7,Franz,Markus,"Leisberg 36, 69124 Heidelberg",NaN,NaN,1,False,NaN,True
7,8,Friedrich,Robert,"Franz-Kruckenberg-Straße 92, 69126 Heidelberg",NaN,NaN,1,True,6.0,True
8,9,Olaf,Borius,"Dürerstraße 10, 69126 Heidelberg",NaN,NaN,1,False,NaN,True
9,10,Clemens,Domi,"Holbeinring 23, 69126 Heidelberg",NaN,NaN,2,False,NaN,False


In [5]:
Afterparty

,Team Nr.,Name 1,Name 2,Adress
0,Afterparty,NaN,NaN,"Im Neuenheimer Feld 304, 69120 Heidelberg"


In [6]:
###setting variables for the latter classification of the Teams into courses ####
Count_teams = 0
for x, _ in Adress_list.iterrows():
	Count_teams += 1

Double_cooks = (3 - int(Count_teams%3))
Double_cooks = Double_cooks if Double_cooks != 3 else 0

Course_width = (int(Count_teams/3) + (1 if Double_cooks !=0 else 0 ))


In [7]:
Double_cooks

2

In [8]:


# Preparing the teams that will be assigned to courses
High_double_cook = Adress_list["will to double"].max()  # This gives the highest value in the "will to double" column



In [9]:
# Filter out the rows with the highest value and then get the next highest value
Second_High_double_cook = Adress_list[Adress_list["will to double"] != High_double_cook]["will to double"].max()


Count_Highest_double_cook = 0
Doublecooks_valuelist = [High_double_cook]  # Start with the highest value


# Iterate through the rows of Adress_list to count how many teams are assigned to double cook
while Count_Highest_double_cook < Double_cooks:
    for _, x in Adress_list.iterrows():
        if x["will to double"] == High_double_cook:
            Count_Highest_double_cook += 1
     #In case there are two necessary doublecooks, but only one team is willing
    if Count_Highest_double_cook != Double_cooks:
        Doublecooks_valuelist.append(Second_High_double_cook)
        continue


In [10]:
Doublecooks_valuelist

[np.int64(5), np.int64(3)]

In [11]:
Adresslist_without_names = Adress_list[["Team Nr.", "Adress", "will to double", "readiness starter", "vegan", "routeplus_vegan"]]
Adresslist_without_names

,Team Nr.,Adress,will to double,readiness starter,vegan,routeplus_vegan
0,1,"Im Neuenheimer Feld 133, 69120 Heidelberg",3,True,True,5.0
1,2,"Im Neuenheimer Feld 522, 69120 Heidelberg",5,True,False,NaN
2,3,"Klostergasse 3, 69123 Heidelberg",1,True,False,NaN
3,4,"Elsterweg 5, 69123 Heidelberg",1,True,False,NaN
4,5,"Kranichweg 88, 69123 Heidelberg",1,True,True,3.0
5,6,"Danziger Straße 16, 69124 Heidelberg",1,True,True,10.0
6,7,"Leisberg 36, 69124 Heidelberg",1,True,False,NaN
7,8,"Franz-Kruckenberg-Straße 92, 69126 Heidelberg",1,True,True,6.0
8,9,"Dürerstraße 10, 69126 Heidelberg",1,True,False,NaN
9,10,"Holbeinring 23, 69126 Heidelberg",2,False,False,NaN


In [12]:
# Adding a delay in the geocode function to avoid hitting the rate limit
def geocode_address(address):
    try:
        time.sleep(1)  # Add a 1-second delay to respect rate limits
        geocode = client.pelias_search(text=address)
        coords = geocode['features'][0]['geometry']['coordinates']
        return coords[0], coords[1]  # longitude, latitude
    except Exception as e:
        print(f"Error geocoding {address}: {e}")
        return None, None


In [13]:
Adresslist_without_names['longitude'], Adresslist_without_names['latitude'] = zip(*Adresslist_without_names['Adress'].apply(geocode_address))

Adresslist_without_names

C:\Users\pfist\AppData\Local\Temp\ipykernel_63024\1818891728.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Adresslist_without_names['longitude'], Adresslist_without_names['latitude'] = zip(*Adresslist_without_names['Adress'].apply(geocode_address))
C:\Users\pfist\AppData\Local\Temp\ipykernel_63024\1818891728.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Adresslist_without_names['longitude'], Adresslist_without_names['latitude'] = zip(*Adresslist_without_names['Adress'].apply(geocode_address))


,Team Nr.,Adress,will to double,readiness starter,vegan,routeplus_vegan,longitude,latitude
0,1,"Im Neuenheimer Feld 133, 69120 Heidelberg",3,True,True,5.0,8.668062,49.412863
1,2,"Im Neuenheimer Feld 522, 69120 Heidelberg",5,True,False,NaN,8.675391,49.420295
2,3,"Klostergasse 3, 69123 Heidelberg",1,True,False,NaN,8.650086,49.423995
3,4,"Elsterweg 5, 69123 Heidelberg",1,True,False,NaN,8.650294,49.404077
4,5,"Kranichweg 88, 69123 Heidelberg",1,True,True,3.0,8.642934,49.394295
5,6,"Danziger Straße 16, 69124 Heidelberg",1,True,True,10.0,8.671389,49.383659
6,7,"Leisberg 36, 69124 Heidelberg",1,True,False,NaN,8.673708,49.383683
7,8,"Franz-Kruckenberg-Straße 92, 69126 Heidelberg",1,True,True,6.0,8.680771,49.382538
8,9,"Dürerstraße 10, 69126 Heidelberg",1,True,False,NaN,8.684597,49.381417
9,10,"Holbeinring 23, 69126 Heidelberg",2,False,False,NaN,8.684331,49.382331


In [14]:
Afterparty['longitude'], Afterparty['latitude'] = zip(*Afterparty['Adress'].apply(geocode_address))

Afterparty

,Team Nr.,Name 1,Name 2,Adress,longitude,latitude
0,Afterparty,NaN,NaN,"Im Neuenheimer Feld 304, 69120 Heidelberg",8.670386,49.415261


In [15]:
# Prepare coordinates for the teams (origins) and afterparty (destination)
Layer_Teams_coords = Adresslist_without_names[['longitude', 'latitude']].apply(lambda row: [row['longitude'], row['latitude']], axis=1).tolist()
Layer_Afterparty_coords = [Afterparty[['longitude', 'latitude']].values[0].tolist()]  # Single destination

# Create the distance matrix request with cycling profile
matrix = client.distance_matrix(
    locations=Layer_Teams_coords + Layer_Afterparty_coords,  # Combine teams and afterparty in one list
    profile='cycling-regular',  # Set profile to cycling-regular for biking
    sources=list(range(len(Layer_Teams_coords))),  # Indices of origins (teams)
    destinations=[len(Layer_Teams_coords)],  # Index of the single destination (afterparty)
    metrics=['distance', 'duration']  # Get both distance and duration
)

# Extract distances and durations for each team
distances = matrix['distances']
durations = matrix['durations']

# Create a DataFrame with Team Numbers and the duration to the afterparty
Distance_to_Afterparty = pd.DataFrame({
    'Team Nr.': Adresslist_without_names['Team Nr.'],
    'Distance_km': [dist[0] / 1000 for dist in distances],  # Convert meters to kilometers
    'Duration_H': [dur[0] / 3600 for dur in durations]  # Convert seconds to hours
})

# Sort by duration in ascending order
Distance_to_Afterparty = Distance_to_Afterparty.sort_values(by='Duration_H', ascending=True)


Distance_to_Afterparty

,Team Nr.,Distance_km,Duration_H
0,1,0.60625,0.048994
1,2,1.03459,0.069594
18,19,2.04118,0.124331
13,14,2.09225,0.128172
14,15,2.28923,0.140075
15,16,3.22368,0.193536
16,17,3.42408,0.213700
17,18,3.52237,0.228511
3,4,3.89951,0.239058
2,3,4.28003,0.254639


In [16]:
# Create a DataFrame with Team Numbers and the duration to the afterparty
Distance_to_Afterparty = pd.DataFrame({
    'Team Nr.': Adresslist_without_names['Team Nr.'],
    'Adress' : Adresslist_without_names['Adress'],
    'will to double' : Adresslist_without_names['will to double'],
    'readiness starter': Adresslist_without_names['readiness starter'],
    'vegan': Adresslist_without_names['vegan'],
    'routeplus_vegan': Adresslist_without_names['routeplus_vegan'],
    'Distance_km': [dist[0] / 1000 for dist in distances],  # Convert meters to kilometers
    'Duration_H': [dur[0] / 3600 for dur in durations],  # Convert seconds to hours
    'latitude' : Adresslist_without_names['latitude'],
    'longitude' : Adresslist_without_names['longitude']
})

# Sort by duration in ascending order
Distance_to_Afterparty = Distance_to_Afterparty.sort_values(by='Duration_H', ascending=True)


Distance_to_Afterparty

,Team Nr.,Adress,will to double,readiness starter,vegan,routeplus_vegan,Distance_km,Duration_H,latitude,longitude
0,1,"Im Neuenheimer Feld 133, 69120 Heidelberg",3,True,True,5.0,0.60625,0.048994,49.412863,8.668062
1,2,"Im Neuenheimer Feld 522, 69120 Heidelberg",5,True,False,NaN,1.03459,0.069594,49.420295,8.675391
18,19,"Mozartstraße 3, 69121 Heidelberg",1,False,True,1.0,2.04118,0.124331,49.421960,8.688278
13,14,"Ringstraße 35/4, 69115 Heidelberg",3,False,False,NaN,2.09225,0.128172,49.402501,8.680582
14,15,"Hildastraße 6, 69115 Heidelberg",3,True,False,NaN,2.28923,0.140075,49.402766,8.682145
15,16,"Gaisbergstraße 29, 69115 Heidelberg",1,True,False,NaN,3.22368,0.193536,49.404258,8.693629
16,17,"Friedrichstraße 10, 69117 Heidelberg",1,True,False,NaN,3.42408,0.213700,49.409785,8.702914
17,18,"Kleine Mantelgasse 17, 69117 Heidelberg",1,True,True,8.0,3.52237,0.228511,49.412582,8.706724
3,4,"Elsterweg 5, 69123 Heidelberg",1,True,False,NaN,3.89951,0.239058,49.404077,8.650294
2,3,"Klostergasse 3, 69123 Heidelberg",1,True,False,NaN,4.28003,0.254639,49.423995,8.650086


In [17]:
Doubles = Distance_to_Afterparty.loc[Distance_to_Afterparty["will to double"].isin(Doublecooks_valuelist)].sort_values(by='will to double', ascending=False)
Doublecooks_Numberlist =[]
for _, x in Doubles[:Double_cooks].iterrows():
    Doublecooks_Numberlist.append(x["Team Nr."])
    

Doublecooks_Numberlist    

[2, 1]

In [18]:
desserts = []
main_courses = []
starters = []

double_courses = []

for _, x in Distance_to_Afterparty[:(Course_width)].iterrows():
        desserts.append(x["Team Nr."])
        if x["Team Nr."] in Doublecooks_Numberlist and len(double_courses) < Double_cooks:
            double_courses.append(x["Team Nr."])
            main_courses.append(x["Team Nr."])
            
desserts


[1, 2, 19, 14, 15, 16, 17]

In [19]:
double_courses

[1, 2]

In [20]:
main_courses

[1, 2]

In [21]:

Dessert_prep = Distance_to_Afterparty.loc[Distance_to_Afterparty["readiness starter"] == True]

for _, x in Dessert_prep[-1:-(Course_width - (Double_cooks-len(double_courses))+1):-1].iterrows():
    starters.append(x["Team Nr."])
    if x["Team Nr."] in Doublecooks_Numberlist and len(double_courses) < Double_cooks:
            double_courses.append(x["Team Nr."])
            main_courses.append(x["Team Nr."])

In [22]:
starters

[12, 13, 5, 11, 9, 8, 6]

In [23]:
double_courses

[1, 2]

In [24]:
main_courses

[1, 2]

In [25]:
already_assigned = starters + desserts + main_courses

for  _, x in Distance_to_Afterparty.loc[Distance_to_Afterparty["Team Nr."].isin(already_assigned) == False].iterrows():
    main_courses.append(x["Team Nr."])
    if x["Team Nr."] in Doublecooks_Numberlist and len(double_courses) < Double_cooks:
            double_courses.append(x["Team Nr."])
            starters.append(x["Team Nr."])
main_courses

[1, 2, 18, 4, 3, 7, 10]

In [26]:
double_courses

[1, 2]

In [27]:
starters

[12, 13, 5, 11, 9, 8, 6]

In [28]:
starters_table = Distance_to_Afterparty.loc[Distance_to_Afterparty["Team Nr."].isin(starters)]
                                            
main_courses_table = Distance_to_Afterparty.loc[Distance_to_Afterparty["Team Nr."].isin(main_courses)]
                                                
desserts_table = Distance_to_Afterparty.loc[Distance_to_Afterparty["Team Nr."].isin(desserts)]
                                
starters_table

,Team Nr.,Adress,will to double,readiness starter,vegan,routeplus_vegan,Distance_km,Duration_H,latitude,longitude
5,6,"Danziger Straße 16, 69124 Heidelberg",1,True,True,10.0,4.46817,0.260158,49.383659,8.671389
7,8,"Franz-Kruckenberg-Straße 92, 69126 Heidelberg",1,True,True,6.0,4.74029,0.277097,49.382538,8.680771
8,9,"Dürerstraße 10, 69126 Heidelberg",1,True,False,NaN,4.90628,0.286319,49.381417,8.684597
10,11,"Holbeinring 7B, 69126 Heidelberg",2,True,False,NaN,5.06985,0.325947,49.382952,8.686384
4,5,"Kranichweg 88, 69123 Heidelberg",1,True,True,3.0,5.52206,0.327525,49.394295,8.642934
12,13,"Curiestraße 7, 69126 Heidelberg",1,True,True,6.0,5.97134,0.346822,49.375703,8.690664
11,12,"Marie-Clauss-Strasse 3, 69126 Heidelberg",1,True,False,NaN,13.97086,0.810886,49.376250,8.769869


In [29]:
main_courses_table

,Team Nr.,Adress,will to double,readiness starter,vegan,routeplus_vegan,Distance_km,Duration_H,latitude,longitude
0,1,"Im Neuenheimer Feld 133, 69120 Heidelberg",3,True,True,5.0,0.60625,0.048994,49.412863,8.668062
1,2,"Im Neuenheimer Feld 522, 69120 Heidelberg",5,True,False,NaN,1.03459,0.069594,49.420295,8.675391
17,18,"Kleine Mantelgasse 17, 69117 Heidelberg",1,True,True,8.0,3.52237,0.228511,49.412582,8.706724
3,4,"Elsterweg 5, 69123 Heidelberg",1,True,False,NaN,3.89951,0.239058,49.404077,8.650294
2,3,"Klostergasse 3, 69123 Heidelberg",1,True,False,NaN,4.28003,0.254639,49.423995,8.650086
6,7,"Leisberg 36, 69124 Heidelberg",1,True,False,NaN,4.39129,0.255889,49.383683,8.673708
9,10,"Holbeinring 23, 69126 Heidelberg",2,False,False,NaN,4.84883,0.289111,49.382331,8.684331


In [30]:
desserts_table

,Team Nr.,Adress,will to double,readiness starter,vegan,routeplus_vegan,Distance_km,Duration_H,latitude,longitude
0,1,"Im Neuenheimer Feld 133, 69120 Heidelberg",3,True,True,5.0,0.60625,0.048994,49.412863,8.668062
1,2,"Im Neuenheimer Feld 522, 69120 Heidelberg",5,True,False,NaN,1.03459,0.069594,49.420295,8.675391
18,19,"Mozartstraße 3, 69121 Heidelberg",1,False,True,1.0,2.04118,0.124331,49.421960,8.688278
13,14,"Ringstraße 35/4, 69115 Heidelberg",3,False,False,NaN,2.09225,0.128172,49.402501,8.680582
14,15,"Hildastraße 6, 69115 Heidelberg",3,True,False,NaN,2.28923,0.140075,49.402766,8.682145
15,16,"Gaisbergstraße 29, 69115 Heidelberg",1,True,False,NaN,3.22368,0.193536,49.404258,8.693629
16,17,"Friedrichstraße 10, 69117 Heidelberg",1,True,False,NaN,3.42408,0.213700,49.409785,8.702914


In [31]:
# Prepare coordinates for the teams (origins) and afterparty (destination)
Layer_starters_coords = starters_table[['Team Nr.', 'longitude', 'latitude']].apply(
    lambda row: {'Team Nr.': row['Team Nr.'], 'coords': [row['longitude'], row['latitude']]}, axis=1
).tolist()

Layer_main_courses_coords = main_courses_table[['Team Nr.', 'longitude', 'latitude']].apply(
    lambda row: {'Team Nr.': row['Team Nr.'], 'coords': [row['longitude'], row['latitude']]}, axis=1
).tolist()

Layer_desserts_coords = desserts_table[['Team Nr.', 'longitude', 'latitude']].apply(
    lambda row: {'Team Nr.': row['Team Nr.'], 'coords': [row['longitude'], row['latitude']]}, axis=1
).tolist()

# Extract coordinates for the matrix calculation
starter_coords = [item['coords'] for item in Layer_starters_coords]
main_course_coords = [item['coords'] for item in Layer_main_courses_coords]
desserts_coords = [item['coords'] for item in Layer_desserts_coords]

In [32]:
# Create the distance matrix request from starters to main courses
matrix = client.distance_matrix(
    locations=starter_coords + main_course_coords,  # Combine starters and main courses
    profile='cycling-regular',  # Set profile to cycling-regular for biking
    sources=list(range(len(starter_coords))),  # Indices of origins (starters)
    destinations=list(range(len(starter_coords), len(starter_coords) + len(main_course_coords))),  # Indices of destinations (main courses)
    metrics=['distance', 'duration']  # Get both distance and duration
)

# Extract distances and durations for the many-to-many matrix
distances = matrix['distances']  # Each source to every destination
durations = matrix['durations']

# Create a DataFrame for the many-to-many relationships
many_to_many_data = []
for i, starter in enumerate(Layer_starters_coords):
    for j, main_course in enumerate(Layer_main_courses_coords):
        many_to_many_data.append({
            'Starter Team Nr.': int(starter['Team Nr.']),
            'Main Course Team Nr.': int(main_course['Team Nr.']),
            'Distance_km': distances[i][j] / 1000,  # Convert meters to kilometers
            'Duration_H': durations[i][j] / 3600  # Convert seconds to hours
        })

Distance_Starters_to_MainCourses = pd.DataFrame(many_to_many_data)
Distance_Starters_to_MainCourses

,Starter Team Nr.,Main Course Team Nr.,Distance_km,Duration_H
0,6,1,4.38856,0.258097
1,6,2,4.59069,0.255539
2,6,18,5.27316,0.313319
3,6,4,3.33386,0.185869
4,6,3,6.25556,0.350353
5,6,7,0.45060,0.025033
6,6,10,2.05257,0.127692
7,8,1,4.66069,0.275036
8,8,2,4.86281,0.272478
9,8,18,4.94777,0.296242


In [33]:
# Create the distance matrix request from main courses to desserts
matrix = client.distance_matrix(
    locations= main_course_coords + desserts_coords,  # Combine starters and main courses
    profile='cycling-regular',  # Set profile to cycling-regular for biking
    sources=list(range(len(main_course_coords))),  # Indices of origins (main_course_coords)
    destinations=list(range(len(main_course_coords), len(main_course_coords) + len(desserts_coords))),  # Indices of destinations (desserts)
    metrics=['distance', 'duration']  # Get both distance and duration
)

# Extract distances and durations for the many-to-many matrix
distances = matrix['distances']  # Each source to every destination
durations = matrix['durations']

# Create a DataFrame for the many-to-many relationships
many_to_many_data = []
for i, main_course in enumerate(Layer_main_courses_coords):
    for j, dessert in enumerate(Layer_desserts_coords):
        many_to_many_data.append({
            'Main Course Team Nr.': int(main_course['Team Nr.']),
            'Dessert Team Nr.': int(dessert['Team Nr.']),
            'Distance_km': distances[i][j] / 1000,  # Convert meters to kilometers
            'Duration_H': durations[i][j] / 3600  # Convert seconds to hours
        })

Distance_MainCourses_to_Desserts = pd.DataFrame(many_to_many_data)
Distance_MainCourses_to_Desserts

,Main Course Team Nr.,Dessert Team Nr.,Distance_km,Duration_H
0,1,1,0.00000,0.000000
1,1,2,1.61935,0.094881
2,1,19,2.74866,0.157522
3,1,14,2.27207,0.133744
4,1,15,2.36905,0.139717
5,1,16,3.19159,0.186467
6,1,17,3.42552,0.208458
7,2,1,1.62872,0.095286
8,2,2,0.00000,0.000000
9,2,19,1.26094,0.071256


In [34]:
# Create the distance matrix request from Starters to desserts
matrix = client.distance_matrix(
    locations= starter_coords + desserts_coords,  # Combine starters and Starters
    profile='cycling-regular',  # Set profile to cycling-regular for biking
    sources=list(range(len(starter_coords))),  # Indices of origins (starter_coords)
    destinations=list(range(len(starter_coords), len(starter_coords) + len(desserts_coords))),  # Indices of destinations (desserts)
    metrics=['distance', 'duration']  # Get both distance and duration
)

# Extract distances and durations for the many-to-many matrix
distances = matrix['distances']  # Each source to every destination
durations = matrix['durations']

# Create a DataFrame for the many-to-many relationships
many_to_many_data = []
for i, starter in enumerate(Layer_starters_coords):
    for j, dessert in enumerate(Layer_desserts_coords):
        many_to_many_data.append({
            'Starter Course Team Nr.': int(starter['Team Nr.']),
            'Dessert Team Nr.': int(dessert['Team Nr.']),
            'Distance_km': distances[i][j] / 1000,  # Convert meters to kilometers
            'Duration_H': durations[i][j] / 3600  # Convert seconds to hours
        })

Distance_Starters_to_Desserts = pd.DataFrame(many_to_many_data)
Distance_Starters_to_Desserts

,Starter Course Team Nr.,Dessert Team Nr.,Distance_km,Duration_H
0,6,1,4.38856,0.258097
1,6,2,4.59069,0.255539
2,6,19,5.44194,0.305792
3,6,14,2.69161,0.149903
4,6,15,2.73103,0.152678
5,6,16,3.41830,0.191194
6,6,17,4.54901,0.262672
7,8,1,4.66069,0.275036
8,8,2,4.86281,0.272478
9,8,19,5.11655,0.288714


In [35]:
Adress_list["Sees 1"] = 0
Adress_list["Sees 2"] = 0
Adress_list["Sees 3"] = 0
Adress_list["Sees 4"] = 0
Adress_list["Sees 5"] = 0
Adress_list["Sees 6"] = 0
Adress_list

,Team Nr.,Name 1,Name 2,Adress,Ring at,Allergies or else,will to double,vegan,routeplus_vegan,readiness starter,Sees 1,Sees 2,Sees 3,Sees 4,Sees 5,Sees 6
0,1,Lukas,Ilaija,"Im Neuenheimer Feld 133, 69120 Heidelberg",NaN,NaN,3,True,5.0,True,0,0,0,0,0,0
1,2,Muhammad,Ilayda,"Im Neuenheimer Feld 522, 69120 Heidelberg",NaN,NaN,5,False,NaN,True,0,0,0,0,0,0
2,3,Sahin,Wolodimr,"Klostergasse 3, 69123 Heidelberg",NaN,NaN,1,False,NaN,True,0,0,0,0,0,0
3,4,Kamala,Micha,"Elsterweg 5, 69123 Heidelberg",NaN,NaN,1,False,NaN,True,0,0,0,0,0,0
4,5,Joe,Donald,"Kranichweg 88, 69123 Heidelberg",NaN,NaN,1,True,3.0,True,0,0,0,0,0,0
5,6,Barak,Angela,"Danziger Straße 16, 69124 Heidelberg",NaN,NaN,1,True,10.0,True,0,0,0,0,0,0
6,7,Franz,Markus,"Leisberg 36, 69124 Heidelberg",NaN,NaN,1,False,NaN,True,0,0,0,0,0,0
7,8,Friedrich,Robert,"Franz-Kruckenberg-Straße 92, 69126 Heidelberg",NaN,NaN,1,True,6.0,True,0,0,0,0,0,0
8,9,Olaf,Borius,"Dürerstraße 10, 69126 Heidelberg",NaN,NaN,1,False,NaN,True,0,0,0,0,0,0
9,10,Clemens,Domi,"Holbeinring 23, 69126 Heidelberg",NaN,NaN,2,False,NaN,False,0,0,0,0,0,0


In [36]:
def assignment(Adress_Table, Distance_Afterparty, Distance_between_Groups, Prioritize_center, Sees_value_first, Sees_value_second, Starts, Targets):
    Distance_Afterparty.sort_values("Duration_H", ascending = Prioritize_center, inplace = True)
    for index, _ in Distance_Afterparty.iterrows():
        
        select_af_team_dist = Distance_Afterparty.loc[index] #this sets the current team and their Afterparty_duration values
        
        select_team_adress = Adress_Table.loc[Adress_Table["Team Nr."] == select_af_team_dist["Team Nr."]] #this sets the current team and their "Sees" values
        select_team_adress_id = select_team_adress["Team Nr."].iloc[0]
        
        select_team_team_dist = Distance_between_Groups.loc[Distance_between_Groups[Starts] == select_af_team_dist["Team Nr."]] #this sets the current team and their respective distances to the targets
        
        
        nogos = [select_team_adress["Sees 1"].iloc[0], select_team_adress["Sees 2"].iloc[0], select_team_adress["Sees 3"].iloc[0], select_team_adress["Sees 4"].iloc[0], select_team_adress["Sees 5"].iloc[0], select_team_adress["Sees 6"].iloc[0]]
        
        if select_team_adress[Sees_value_first].iloc[0] != 0: #skips already assigned teams (might be obsolete because carried over from old code)
            continue

        if select_team_team_dist[Starts].isin(select_team_team_dist[Targets]).any(): #skips teams who will be hosts next step
            continue
        
        
        relevant_targets = Distance_between_Groups.loc[Distance_between_Groups[Starts] == select_team_adress_id] #selects the Targets for iterating 
        relevant_targets.sort_values("Duration_H", ascending = True, inplace = True) #sorts Targets by their respective distance to the Team
        
        #iterates over the selected targets to find suitable host
        for _, target in relevant_targets.iterrows():
            
            target_id = target[Targets] #selects the Team Id of the selected Target
            
            if target_id in nogos:
                continue
            
            
            target_adress_id = Adress_Table.loc[Adress_Table["Team Nr."] == target_id]
            
            if target_adress_id[Sees_value_first].iloc[0] == 0: # sets the "sees 1, sees 3, sees 5" values
                Adress_Table.loc[Adress_Table["Team Nr."] == target_id, Sees_value_first] = select_af_team_dist["Team Nr."]
                Adress_Table.loc[Adress_Table["Team Nr."] == select_af_team_dist["Team Nr."], Sees_value_first] = target_id
                break
              
                
            elif target_adress_id[Sees_value_second].iloc[0] == 0: #sets the "sees 1, sees 3, sees 5" value of guest and "sees 2, sees 4, sees 6" value of host
                #hotfix
                if target_adress_id[Sees_value_first].iloc[0] in nogos:
                    continue
                else:
                    Adress_Table.loc[Adress_Table["Team Nr."] == target_id, Sees_value_second] = select_af_team_dist["Team Nr."]
                    Adress_Table.loc[Adress_Table["Team Nr."] == select_af_team_dist["Team Nr."], Sees_value_first] = target_id

                    # Assign third team for "sees second"
                    the_third_team = target_adress_id[Sees_value_first].iloc[0]
                    Adress_Table.loc[Adress_Table["Team Nr."] == select_af_team_dist["Team Nr."], Sees_value_second] = the_third_team
                    Adress_Table.loc[Adress_Table["Team Nr."] == the_third_team, Sees_value_second] = select_af_team_dist["Team Nr."]
                    break

                
            



In [37]:
assignment(Adress_list, main_courses_table, Distance_Starters_to_MainCourses, True, "Sees 1", "Sees 2", "Main Course Team Nr.", "Starter Team Nr.")

### def assignment(Adress_Table, Distance_Afterparty, Distance_between_Groups, Prioritize_center, Sees_value_first, Sees_value_second, Starts, Targets):

Adress_list

C:\Users\pfist\AppData\Local\Temp\ipykernel_63024\2597795099.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Distance_Afterparty.sort_values("Duration_H", ascending = Prioritize_center, inplace = True)
C:\Users\pfist\AppData\Local\Temp\ipykernel_63024\2597795099.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  relevant_targets.sort_values("Duration_H", ascending = True, inplace = True) #sorts Targets by their respective distance to the Team
C:\Users\pfist\AppData\Local\Temp\ipykernel_63024\2597795099.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the cavea

,Team Nr.,Name 1,Name 2,Adress,Ring at,Allergies or else,will to double,vegan,routeplus_vegan,readiness starter,Sees 1,Sees 2,Sees 3,Sees 4,Sees 5,Sees 6
0,1,Lukas,Ilaija,"Im Neuenheimer Feld 133, 69120 Heidelberg",NaN,NaN,3,True,5.0,True,6,2,0,0,0,0
1,2,Muhammad,Ilayda,"Im Neuenheimer Feld 522, 69120 Heidelberg",NaN,NaN,5,False,NaN,True,6,1,0,0,0,0
2,3,Sahin,Wolodimr,"Klostergasse 3, 69123 Heidelberg",NaN,NaN,1,False,NaN,True,5,4,0,0,0,0
3,4,Kamala,Micha,"Elsterweg 5, 69123 Heidelberg",NaN,NaN,1,False,NaN,True,5,3,0,0,0,0
4,5,Joe,Donald,"Kranichweg 88, 69123 Heidelberg",NaN,NaN,1,True,3.0,True,4,3,0,0,0,0
5,6,Barak,Angela,"Danziger Straße 16, 69124 Heidelberg",NaN,NaN,1,True,10.0,True,1,2,0,0,0,0
6,7,Franz,Markus,"Leisberg 36, 69124 Heidelberg",NaN,NaN,1,False,NaN,True,8,18,0,0,0,0
7,8,Friedrich,Robert,"Franz-Kruckenberg-Straße 92, 69126 Heidelberg",NaN,NaN,1,True,6.0,True,18,7,0,0,0,0
8,9,Olaf,Borius,"Dürerstraße 10, 69126 Heidelberg",NaN,NaN,1,False,NaN,True,10,0,0,0,0,0
9,10,Clemens,Domi,"Holbeinring 23, 69126 Heidelberg",NaN,NaN,2,False,NaN,False,9,0,0,0,0,0


In [38]:
assignment(Adress_list, desserts_table, Distance_Starters_to_Desserts, True, "Sees 1", "Sees 2", "Dessert Team Nr.", "Starter Course Team Nr.")
Adress_list

C:\Users\pfist\AppData\Local\Temp\ipykernel_63024\2597795099.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Distance_Afterparty.sort_values("Duration_H", ascending = Prioritize_center, inplace = True)
C:\Users\pfist\AppData\Local\Temp\ipykernel_63024\2597795099.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  relevant_targets.sort_values("Duration_H", ascending = True, inplace = True) #sorts Targets by their respective distance to the Team
C:\Users\pfist\AppData\Local\Temp\ipykernel_63024\2597795099.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the cavea

,Team Nr.,Name 1,Name 2,Adress,Ring at,Allergies or else,will to double,vegan,routeplus_vegan,readiness starter,Sees 1,Sees 2,Sees 3,Sees 4,Sees 5,Sees 6
0,1,Lukas,Ilaija,"Im Neuenheimer Feld 133, 69120 Heidelberg",NaN,NaN,3,True,5.0,True,6,2,0,0,0,0
1,2,Muhammad,Ilayda,"Im Neuenheimer Feld 522, 69120 Heidelberg",NaN,NaN,5,False,NaN,True,6,1,0,0,0,0
2,3,Sahin,Wolodimr,"Klostergasse 3, 69123 Heidelberg",NaN,NaN,1,False,NaN,True,5,4,0,0,0,0
3,4,Kamala,Micha,"Elsterweg 5, 69123 Heidelberg",NaN,NaN,1,False,NaN,True,5,3,0,0,0,0
4,5,Joe,Donald,"Kranichweg 88, 69123 Heidelberg",NaN,NaN,1,True,3.0,True,4,3,0,0,0,0
5,6,Barak,Angela,"Danziger Straße 16, 69124 Heidelberg",NaN,NaN,1,True,10.0,True,1,2,0,0,0,0
6,7,Franz,Markus,"Leisberg 36, 69124 Heidelberg",NaN,NaN,1,False,NaN,True,8,18,0,0,0,0
7,8,Friedrich,Robert,"Franz-Kruckenberg-Straße 92, 69126 Heidelberg",NaN,NaN,1,True,6.0,True,18,7,0,0,0,0
8,9,Olaf,Borius,"Dürerstraße 10, 69126 Heidelberg",NaN,NaN,1,False,NaN,True,10,19,0,0,0,0
9,10,Clemens,Domi,"Holbeinring 23, 69126 Heidelberg",NaN,NaN,2,False,NaN,False,9,19,0,0,0,0


In [39]:
Copy_Star_Dess = Distance_Starters_to_Desserts
Copy_Star_Dess

,Starter Course Team Nr.,Dessert Team Nr.,Distance_km,Duration_H
0,6,1,4.38856,0.258097
1,6,2,4.59069,0.255539
2,6,19,5.44194,0.305792
3,6,14,2.69161,0.149903
4,6,15,2.73103,0.152678
5,6,16,3.41830,0.191194
6,6,17,4.54901,0.262672
7,8,1,4.66069,0.275036
8,8,2,4.86281,0.272478
9,8,19,5.11655,0.288714


In [40]:
def overwrite(guest_distances, host_distances, Adress_Table, guest_start, guest_target, host_start, host_target, Sees_first):
    for index, row in guest_distances.iterrows():
        #copy the distances from the hosts to the next course onto the guests
        guest_team = guest_distances.loc[index]
        guest_team_in_Adresses = Adress_Table.loc[Adress_Table["Team Nr."] == guest_team[guest_start]]


        if guest_team_in_Adresses[Sees_first].iloc[0] != guest_team[guest_target]:
            continue

        guest_team_to = guest_team[guest_target]  # ID of host team
        guest_team_from = guest_team[guest_start]  # ID of guest team

        relevant_guests = guest_distances[guest_distances[guest_start] == guest_team_from]
        relevant_hosts = host_distances[host_distances[host_start] == guest_team_to]

        for (index1,_), (index2,__) in zip(relevant_guests.iterrows(), relevant_hosts.iterrows()):

            route_host = host_distances.loc[index2, "Duration_H"]
            target_host = host_distances.loc[index2, host_target]

            guest_distances.loc[index1, guest_target] = target_host
            guest_distances.loc[index1, "Duration_H"] = route_host

In [41]:
overwrite(Copy_Star_Dess, Distance_Starters_to_MainCourses, Adress_list, "Dessert Team Nr.", "Starter Course Team Nr.", "Starter Team Nr.", "Main Course Team Nr.", "Sees 1")
#write distances of starter Hosts to their Dessert Team Guests


In [42]:
Copy_Star_Dess.sort_values(by = ["Dessert Team Nr.", "Starter Course Team Nr."], ascending = [True, True], inplace = True)
Copy_Star_Dess["Main Course Team Nr."] = Copy_Star_Dess["Starter Course Team Nr."]
del Copy_Star_Dess["Starter Course Team Nr."]

Copy_Star_Dess

,Dessert Team Nr.,Distance_km,Duration_H,Main Course Team Nr.
0,1,4.38856,0.258097,1
7,1,4.66069,0.255539,2
28,1,5.44245,0.350353,3
21,1,4.99024,0.185869,4
35,1,5.89173,0.025033,7
42,1,13.95785,0.127692,10
14,1,4.82667,0.313319,18
1,2,4.59069,0.258097,1
8,2,4.86281,0.255539,2
29,2,5.64458,0.350353,3


In [43]:
assignment(Adress_list, desserts_table, Copy_Star_Dess, False, "Sees 3", "Sees 4", "Dessert Team Nr.", "Main Course Team Nr.")
#assign the dessert Teams to their Main Course Hosts
Adress_list

C:\Users\pfist\AppData\Local\Temp\ipykernel_63024\2597795099.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Distance_Afterparty.sort_values("Duration_H", ascending = Prioritize_center, inplace = True)
C:\Users\pfist\AppData\Local\Temp\ipykernel_63024\2597795099.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  relevant_targets.sort_values("Duration_H", ascending = True, inplace = True) #sorts Targets by their respective distance to the Team
C:\Users\pfist\AppData\Local\Temp\ipykernel_63024\2597795099.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the cavea

,Team Nr.,Name 1,Name 2,Adress,Ring at,Allergies or else,will to double,vegan,routeplus_vegan,readiness starter,Sees 1,Sees 2,Sees 3,Sees 4,Sees 5,Sees 6
0,1,Lukas,Ilaija,"Im Neuenheimer Feld 133, 69120 Heidelberg",NaN,NaN,3,True,5.0,True,6,2,0,0,0,0
1,2,Muhammad,Ilayda,"Im Neuenheimer Feld 522, 69120 Heidelberg",NaN,NaN,5,False,NaN,True,6,1,0,0,0,0
2,3,Sahin,Wolodimr,"Klostergasse 3, 69123 Heidelberg",NaN,NaN,1,False,NaN,True,5,4,0,0,0,0
3,4,Kamala,Micha,"Elsterweg 5, 69123 Heidelberg",NaN,NaN,1,False,NaN,True,5,3,19,0,0,0
4,5,Joe,Donald,"Kranichweg 88, 69123 Heidelberg",NaN,NaN,1,True,3.0,True,4,3,0,0,0,0
5,6,Barak,Angela,"Danziger Straße 16, 69124 Heidelberg",NaN,NaN,1,True,10.0,True,1,2,0,0,0,0
6,7,Franz,Markus,"Leisberg 36, 69124 Heidelberg",NaN,NaN,1,False,NaN,True,8,18,15,14,0,0
7,8,Friedrich,Robert,"Franz-Kruckenberg-Straße 92, 69126 Heidelberg",NaN,NaN,1,True,6.0,True,18,7,0,0,0,0
8,9,Olaf,Borius,"Dürerstraße 10, 69126 Heidelberg",NaN,NaN,1,False,NaN,True,10,19,0,0,0,0
9,10,Clemens,Domi,"Holbeinring 23, 69126 Heidelberg",NaN,NaN,2,False,NaN,False,9,19,17,16,0,0


In [44]:
assignment(Adress_list, starters_table, Distance_Starters_to_MainCourses, True, "Sees 3", "Sees 4", "Starter Team Nr.", "Main Course Team Nr.")
#assign the starter Teams to their Main Course Hosts
Adress_list

C:\Users\pfist\AppData\Local\Temp\ipykernel_63024\2597795099.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Distance_Afterparty.sort_values("Duration_H", ascending = Prioritize_center, inplace = True)
C:\Users\pfist\AppData\Local\Temp\ipykernel_63024\2597795099.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  relevant_targets.sort_values("Duration_H", ascending = True, inplace = True) #sorts Targets by their respective distance to the Team
C:\Users\pfist\AppData\Local\Temp\ipykernel_63024\2597795099.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the cavea

,Team Nr.,Name 1,Name 2,Adress,Ring at,Allergies or else,will to double,vegan,routeplus_vegan,readiness starter,Sees 1,Sees 2,Sees 3,Sees 4,Sees 5,Sees 6
0,1,Lukas,Ilaija,"Im Neuenheimer Feld 133, 69120 Heidelberg",NaN,NaN,3,True,5.0,True,6,2,11,5,0,0
1,2,Muhammad,Ilayda,"Im Neuenheimer Feld 522, 69120 Heidelberg",NaN,NaN,5,False,NaN,True,6,1,8,9,0,0
2,3,Sahin,Wolodimr,"Klostergasse 3, 69123 Heidelberg",NaN,NaN,1,False,NaN,True,5,4,0,0,0,0
3,4,Kamala,Micha,"Elsterweg 5, 69123 Heidelberg",NaN,NaN,1,False,NaN,True,5,3,19,6,0,0
4,5,Joe,Donald,"Kranichweg 88, 69123 Heidelberg",NaN,NaN,1,True,3.0,True,4,3,1,11,0,0
5,6,Barak,Angela,"Danziger Straße 16, 69124 Heidelberg",NaN,NaN,1,True,10.0,True,1,2,4,19,0,0
6,7,Franz,Markus,"Leisberg 36, 69124 Heidelberg",NaN,NaN,1,False,NaN,True,8,18,15,14,0,0
7,8,Friedrich,Robert,"Franz-Kruckenberg-Straße 92, 69126 Heidelberg",NaN,NaN,1,True,6.0,True,18,7,2,9,0,0
8,9,Olaf,Borius,"Dürerstraße 10, 69126 Heidelberg",NaN,NaN,1,False,NaN,True,10,19,2,8,0,0
9,10,Clemens,Domi,"Holbeinring 23, 69126 Heidelberg",NaN,NaN,2,False,NaN,False,9,19,17,16,0,0


In [45]:
Copy_Star_Main = Distance_Starters_to_MainCourses

Copy_Star_Main


,Starter Team Nr.,Main Course Team Nr.,Distance_km,Duration_H
0,6,1,4.38856,0.258097
1,6,2,4.59069,0.255539
2,6,18,5.27316,0.313319
3,6,4,3.33386,0.185869
4,6,3,6.25556,0.350353
5,6,7,0.45060,0.025033
6,6,10,2.05257,0.127692
7,8,1,4.66069,0.275036
8,8,2,4.86281,0.272478
9,8,18,4.94777,0.296242


In [46]:
overwrite(Copy_Star_Main, Distance_MainCourses_to_Desserts, Adress_list, "Starter Team Nr.", "Main Course Team Nr.", "Main Course Team Nr.", "Dessert Team Nr.", "Sees 3")
Copy_Star_Main["Dessert Team Nr."] = Copy_Star_Main["Main Course Team Nr."]
del Copy_Star_Main["Main Course Team Nr."]
Copy_Star_Main

,Starter Team Nr.,Distance_km,Duration_H,Dessert Team Nr.
0,6,4.38856,0.236997,1
1,6,4.59069,0.234439,2
2,6,5.27316,0.367986,19
3,6,3.33386,0.212094,14
4,6,6.25556,0.214869,15
5,6,0.45060,0.279947,16
6,6,2.05257,0.324864,17
7,8,4.66069,0.095286,1
8,8,4.86281,0.000000,2
9,8,4.94777,0.071256,19


In [47]:
assignment(Adress_list, main_courses_table, Distance_MainCourses_to_Desserts, True, "Sees 5", "Sees 6", "Main Course Team Nr.", "Dessert Team Nr.")
#assign the Main Course Teams to their Dessert Hosts
Adress_list

C:\Users\pfist\AppData\Local\Temp\ipykernel_63024\2597795099.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Distance_Afterparty.sort_values("Duration_H", ascending = Prioritize_center, inplace = True)
C:\Users\pfist\AppData\Local\Temp\ipykernel_63024\2597795099.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  relevant_targets.sort_values("Duration_H", ascending = True, inplace = True) #sorts Targets by their respective distance to the Team
C:\Users\pfist\AppData\Local\Temp\ipykernel_63024\2597795099.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the cavea

,Team Nr.,Name 1,Name 2,Adress,Ring at,Allergies or else,will to double,vegan,routeplus_vegan,readiness starter,Sees 1,Sees 2,Sees 3,Sees 4,Sees 5,Sees 6
0,1,Lukas,Ilaija,"Im Neuenheimer Feld 133, 69120 Heidelberg",NaN,NaN,3,True,5.0,True,6,2,11,5,0,0
1,2,Muhammad,Ilayda,"Im Neuenheimer Feld 522, 69120 Heidelberg",NaN,NaN,5,False,NaN,True,6,1,8,9,0,0
2,3,Sahin,Wolodimr,"Klostergasse 3, 69123 Heidelberg",NaN,NaN,1,False,NaN,True,5,4,0,0,15,0
3,4,Kamala,Micha,"Elsterweg 5, 69123 Heidelberg",NaN,NaN,1,False,NaN,True,5,3,19,6,14,10
4,5,Joe,Donald,"Kranichweg 88, 69123 Heidelberg",NaN,NaN,1,True,3.0,True,4,3,1,11,0,0
5,6,Barak,Angela,"Danziger Straße 16, 69124 Heidelberg",NaN,NaN,1,True,10.0,True,1,2,4,19,0,0
6,7,Franz,Markus,"Leisberg 36, 69124 Heidelberg",NaN,NaN,1,False,NaN,True,8,18,15,14,16,0
7,8,Friedrich,Robert,"Franz-Kruckenberg-Straße 92, 69126 Heidelberg",NaN,NaN,1,True,6.0,True,18,7,2,9,0,0
8,9,Olaf,Borius,"Dürerstraße 10, 69126 Heidelberg",NaN,NaN,1,False,NaN,True,10,19,2,8,0,0
9,10,Clemens,Domi,"Holbeinring 23, 69126 Heidelberg",NaN,NaN,2,False,NaN,False,9,19,17,16,14,4


In [48]:
assignment(Adress_list, starters_table, Copy_Star_Main, False, "Sees 5", "Sees 6", "Starter Team Nr.", "Dessert Team Nr.")
#assign the Starter Teams to their Dessert Hosts
Adress_list

C:\Users\pfist\AppData\Local\Temp\ipykernel_63024\2597795099.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Distance_Afterparty.sort_values("Duration_H", ascending = Prioritize_center, inplace = True)
C:\Users\pfist\AppData\Local\Temp\ipykernel_63024\2597795099.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  relevant_targets.sort_values("Duration_H", ascending = True, inplace = True) #sorts Targets by their respective distance to the Team
C:\Users\pfist\AppData\Local\Temp\ipykernel_63024\2597795099.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the cavea

,Team Nr.,Name 1,Name 2,Adress,Ring at,Allergies or else,will to double,vegan,routeplus_vegan,readiness starter,Sees 1,Sees 2,Sees 3,Sees 4,Sees 5,Sees 6
0,1,Lukas,Ilaija,"Im Neuenheimer Feld 133, 69120 Heidelberg",NaN,NaN,3,True,5.0,True,6,2,11,5,9,0
1,2,Muhammad,Ilayda,"Im Neuenheimer Feld 522, 69120 Heidelberg",NaN,NaN,5,False,NaN,True,6,1,8,9,5,0
2,3,Sahin,Wolodimr,"Klostergasse 3, 69123 Heidelberg",NaN,NaN,1,False,NaN,True,5,4,0,0,15,11
3,4,Kamala,Micha,"Elsterweg 5, 69123 Heidelberg",NaN,NaN,1,False,NaN,True,5,3,19,6,14,10
4,5,Joe,Donald,"Kranichweg 88, 69123 Heidelberg",NaN,NaN,1,True,3.0,True,4,3,1,11,2,0
5,6,Barak,Angela,"Danziger Straße 16, 69124 Heidelberg",NaN,NaN,1,True,10.0,True,1,2,4,19,17,18
6,7,Franz,Markus,"Leisberg 36, 69124 Heidelberg",NaN,NaN,1,False,NaN,True,8,18,15,14,16,12
7,8,Friedrich,Robert,"Franz-Kruckenberg-Straße 92, 69126 Heidelberg",NaN,NaN,1,True,6.0,True,18,7,2,9,19,13
8,9,Olaf,Borius,"Dürerstraße 10, 69126 Heidelberg",NaN,NaN,1,False,NaN,True,10,19,2,8,1,0
9,10,Clemens,Domi,"Holbeinring 23, 69126 Heidelberg",NaN,NaN,2,False,NaN,False,9,19,17,16,14,4


In [49]:
Adress_list

,Team Nr.,Name 1,Name 2,Adress,Ring at,Allergies or else,will to double,vegan,routeplus_vegan,readiness starter,Sees 1,Sees 2,Sees 3,Sees 4,Sees 5,Sees 6
0,1,Lukas,Ilaija,"Im Neuenheimer Feld 133, 69120 Heidelberg",NaN,NaN,3,True,5.0,True,6,2,11,5,9,0
1,2,Muhammad,Ilayda,"Im Neuenheimer Feld 522, 69120 Heidelberg",NaN,NaN,5,False,NaN,True,6,1,8,9,5,0
2,3,Sahin,Wolodimr,"Klostergasse 3, 69123 Heidelberg",NaN,NaN,1,False,NaN,True,5,4,0,0,15,11
3,4,Kamala,Micha,"Elsterweg 5, 69123 Heidelberg",NaN,NaN,1,False,NaN,True,5,3,19,6,14,10
4,5,Joe,Donald,"Kranichweg 88, 69123 Heidelberg",NaN,NaN,1,True,3.0,True,4,3,1,11,2,0
5,6,Barak,Angela,"Danziger Straße 16, 69124 Heidelberg",NaN,NaN,1,True,10.0,True,1,2,4,19,17,18
6,7,Franz,Markus,"Leisberg 36, 69124 Heidelberg",NaN,NaN,1,False,NaN,True,8,18,15,14,16,12
7,8,Friedrich,Robert,"Franz-Kruckenberg-Straße 92, 69126 Heidelberg",NaN,NaN,1,True,6.0,True,18,7,2,9,19,13
8,9,Olaf,Borius,"Dürerstraße 10, 69126 Heidelberg",NaN,NaN,1,False,NaN,True,10,19,2,8,1,0
9,10,Clemens,Domi,"Holbeinring 23, 69126 Heidelberg",NaN,NaN,2,False,NaN,False,9,19,17,16,14,4


In [50]:
Starters_Sentences = []
Main_Courses_Sentences = []
Desserts_Sentences = []

for (index1,_), (index2,__), (index3, ___) in zip(starters_table.iterrows(), main_courses_table.iterrows(), desserts_table.iterrows()):
    This_Starter = starters_table.loc[index1]
    This_MainCourse = main_courses_table.loc[index2]
    This_Dessert = desserts_table.loc[index3]
    
    Star_in_Ad = Adress_list[Adress_list["Team Nr."] == This_Starter["Team Nr."]]
    Main_in_Ad = Adress_list[Adress_list["Team Nr."] == This_MainCourse["Team Nr."]]
    Des_in_Ad = Adress_list[Adress_list["Team Nr."] == This_Dessert["Team Nr."]]
    
    Sta_Sen = f'Team Nr.{Star_in_Ad["Team Nr."].iloc[0]} (Starter) hosts the Teams {Star_in_Ad["Sees 1"].iloc[0]} and {Star_in_Ad["Sees 2"].iloc[0]}.'
    Mai_Sen = f'Team Nr.{Main_in_Ad["Team Nr."].iloc[0]} (Main Course) hosts the Teams {Main_in_Ad["Sees 3"].iloc[0]} and {Main_in_Ad["Sees 4"].iloc[0]}.'
    Des_Sen = f'Team Nr.{Des_in_Ad["Team Nr."].iloc[0]} (Dessert) hosts the Teams {Des_in_Ad["Sees 5"].iloc[0]} and {Des_in_Ad["Sees 6"].iloc[0]}.'
    
    Starters_Sentences.append(Sta_Sen)
    Main_Courses_Sentences.append(Mai_Sen)
    Desserts_Sentences.append(Des_Sen)
    
Starters_Sentences = set(Starters_Sentences)
Main_Courses_Sentences = set(Main_Courses_Sentences)
Desserts_Sentences = set(Desserts_Sentences)


with open('overview.txt', 'w') as file:
    file.write('Starters\n\n')
    
    for Sentence in Starters_Sentences:
        file.write(f'{Sentence}\n\n')
        
    file.write('Main Courses\n\n')
    
    for Sentence in Main_Courses_Sentences:
        file.write(f'{Sentence}\n\n')
        
    file.write('Desserts\n\n')
    
    for Sentence in Desserts_Sentences:
        file.write(f'{Sentence}\n\n')

In [ ]:
lang = input("please select a language for mail creation\n|1---english\n|2---deutsch\n-------")

Time_Sta = input("please type the time the starters should begin\n\n")
Time_Mai = input("please type the time the main courses should begin\n\n")
Time_Des = input("please type the time the desserts should begin\n\n")

Time_Afterparty = input("please type the time the Afterparty is planned\n\n")
Afterparty_Drinks = input("1----drinks will be provided by the location for a price\n\n2----drinks will be provided for free by us\n\n")

Orga_Name = input("please type the Name(s) of the person(s) responsible for making heat of the Moment adjustments and information delegation\n\n")
Orga_Tel = input("please type the Tel.-Number(s) of the person(s) responsible for making heat of the Moment adjustments and information delegation\n\n")

Awareness = input("will there be an Awareness Team?\n\n1-----yes\n\n2-----no\n\n")
if Awareness == "1":
    Aw_Tel = input("what is their Tel.-Nr.?")


In [ ]:
if lang == '2':
    ###the information regarding the Afterparty
    if Afterparty_Drinks == '1':
        Ausklang = (
            f'Ab ca. {Time_Afterparty} Uhr treffen wir uns an folgender Adresse: {Afterparty["Adress"].iloc[0]}'
            f'zum gemütlichen Ausklang des Abends. Hier könnt ihr eure Rezepte austauschen oder übriggebliebene Speisen mitbringen.'
            f'Es wird Getränke auf Spendenbasis geben.\n\n\n\n'
        )
        
    else:
        Ausklang = (
            f'Ab ca. {Time_Afterparty} Uhr treffen wir uns an folgender Adresse: {Afterparty["Adress"].iloc[0]}'
            f'zum gemütlichen Ausklang des Abends. Hier könnt ihr eure Rezepte austauschen oder übriggebliebene Speisen mitbringen.\n\n\n\n'
        )
        
        
    if Awareness == '1':    
        Schlusswort = (
            f'Bitte gebt eurem/r Kochpartner/in Bescheid, da diese Email nur an eine Person geht.\n\n'
            f'Bei Notfällen/Unstimmigkeiten an dem Abend meldet euch bei {Orga_Name} unter {Orga_Tel}\n\n'
            f'Die Handynummer des A-Teams lautet: {Aw_Tel}\n\n'
            f'Wichtig: Falls ihr krankheitsbedingt ausfallt und selber keinen Ersatz findet gebt uns bitte schnellstmöglich Bescheid.\n\n'
        )
    else:    
        Schlusswort = (
            f'Bitte gebt eurem/r Kochpartner/in Bescheid, da diese Email nur an eine Person geht.\n\n'
            f'Bei Notfällen/Unstimmigkeiten an dem Abend meldet euch bei {Orga_Name} unter {Orga_Tel}\n\n'
            f'Wichtig: Falls ihr krankheitsbedingt ausfallt und selber keinen Ersatz findet gebt uns bitte schnellstmöglich Bescheid.\n\n'
        )
    for _, Team in Adress_list.iterrows():
        Anrede = f'Hallo {Team["Name 1"]} und {Team["Name 2"]}, \n\nVielen Dank für eure Anmeldung\n\n. Hier euer Plan für den Abend: \n\n\n\n'
        if Team["Team Nr."] in starters:
            
            if Team["Sees 1"] != 0:
                guest_1 = Adress_list.loc[Adress_list["Team Nr."] == Team["Sees 1"]].iloc[0]
            else: 
                guest_1 = Team
            if Team["Sees 2"] != 0:
                guest_2 = Adress_list.loc[Adress_list["Team Nr."] == Team["Sees 2"]].iloc[0]
            else: 
                guest_2 = Team
            
            Vorspeise = (
                f"Ihr dürft gleich am Anfang aktiv sein und euren Gästen eine unvergessliche Vorspeise servieren "
                f"(Essgewohnheiten: {guest_1['Allergies or else']} UND {guest_2['Allergies or else']}).\n\n"
                f"Ob ihr zu eurem Gericht ein stilles Mineralwasser, ein fancy Aperitif oder ein Gläschen süffigen Wein serviert, "
                f"bleibt euch natürlich selbst überlassen.\n\n"
            )
            
            Zeitplan = (
                f"Die Zeitplanübersicht:\n\n\n\n{Time_Sta}-----Vorspeise\n\n\n\n{Time_Mai}-----Hauptspeise\n\n\n\n{Time_Des}-----Nachspeise"
                f"Denkt bitte bei euren Speisen daran, sie entweder schon vorzubereiten oder nicht zu aufwendig zu gestalten, da um {Time_Sta}"
                f"eure Gäste eintreffen werden. Außerdem solltet ihr den Fahrtweg miteinplanen, sodass ihr jeweils zur angegebenen Uhrzeit beim nächsten Team seid."
            )
                
        else:
            host = Adress_list.loc[Adress_list["Team Nr."] == Team["Sees 1"]].iloc[0]                              
            Vorspeise = (
                f"Den Abend beginnt ihr bei {host['Name 1']} und {host['Name 2']} ({host['Adress']}), {host['Ring at']} mit einer genialen Vorspeise.\n\n"
            )
                                          
                                          

        if Team["Team Nr."] in main_courses:
            
            if Team["Sees 3"] != 0:
                guest_1 = Adress_list.loc[Adress_list["Team Nr."] == Team["Sees 3"]].iloc[0]
            else: 
                guest_1 = Team
            if Team["Sees 4"] != 0:
                guest_2 = Adress_list.loc[Adress_list["Team Nr."] == Team["Sees 4"]].iloc[0]
            else: 
                guest_2 = Team
            
            Hauptspeise = (
                f"Danach seid ihr selbst an der Reihe euren Gästen eine unvergessliche Hauptspeise anzubieten."
                f"(Essgewohnheiten: {guest_1['Allergies or else']} UND {guest_2['Allergies or else']}).\n\n"
                f"Ob ihr zu eurem Gericht ein stilles Mineralwasser, ein fancy Aperitif oder ein Gläschen süffigen Wein serviert, "
                f"bleibt euch natürlich selbst überlassen.\n\n"
            )
            
            Zeitplan = (
                f"Die Zeitplanübersicht:\n\n\n\n{Time_Sta}-----Vorspeise\n\n\n\n{Time_Mai}-----Hauptspeise\n\n\n\n{Time_Des}-----Nachspeise"
                f"Denkt bitte bei euren Speisen daran, sie entweder schon vorzubereiten oder nicht zu aufwendig zu gestalten, da um {Time_Mai}"
                f"eure Gäste eintreffen werden. Außerdem solltet ihr den Fahrtweg miteinplanen, sodass ihr jeweils zur angegebenen Uhrzeit beim nächsten Team seid."
            )
        else:
            host = Adress_list.loc[Adress_list["Team Nr."] == Team["Sees 3"]].iloc[0]                              
            Hauptspeise = (
                f"Im Anschluss daran beglücken euch {host['Name 1']} und {host['Name 2']} ({host['Adress']}), {host['Ring at']} mit einer grandiosen Hauptspeise.\n\n"
            )
            
            
        if Team["Team Nr."] in desserts:
            
            if Team["Sees 5"] != 0:
                guest_1 = Adress_list.loc[Adress_list["Team Nr."] == Team["Sees 5"]].iloc[0]
            else: 
                guest_1 = Team
            if Team["Sees 6"] != 0:
                guest_2 = Adress_list.loc[Adress_list["Team Nr."] == Team["Sees 6"]].iloc[0]
            else: 
                guest_2 = Team
            
            Nachspeise = (
                f"Zum Schluss seid ihr selbst an der Reihe euren Gästen eine leckere Nachspeise zu servieren."
                f"(Essgewohnheiten: {guest_1['Allergies or else']} UND {guest_2['Allergies or else']}).\n\n"
                f"Ob ihr zu eurem Gericht ein stilles Mineralwasser, ein fancy Aperitif oder ein Gläschen süffigen Wein serviert, "
                f"bleibt euch natürlich selbst überlassen.\n\n"
            )
            
            Zeitplan = (
                f"Die Zeitplanübersicht:\n\n\n\n{Time_Sta}-----Vorspeise\n\n\n\n{Time_Mai}-----Hauptspeise\n\n\n\n{Time_Des}-----Nachspeise"
                f"Denkt bitte bei euren Speisen daran, sie entweder schon vorzubereiten oder nicht zu aufwendig zu gestalten, da um {Time_Des}"
                f"eure Gäste eintreffen werden. Außerdem solltet ihr den Fahrtweg miteinplanen, sodass ihr jeweils zur angegebenen Uhrzeit beim nächsten Team seid."
            )
        else:
            host = Adress_list.loc[Adress_list["Team Nr."] == Team["Sees 5"]].iloc[0]                              
            Nachspeise = (
                f"Zur Nachspeise servieren euch {host['Name 1']} und {host['Name 2']} ({host['Adress']}), {host['Ring at']} etwas ganz Traumhaftes für den letzten Platz im Magen.\n\n"
            )
        with open(f'Mail Team Nr.{Team["Team Nr."]}.txt', 'w') as file:
            file.write(f'{Anrede} {Vorspeise} {Hauptspeise} {Nachspeise} {Zeitplan} {Ausklang} {Schlusswort}')